In [1]:
import glob
import os
import pandas as pd

In [2]:
# 指定したファイルパスのcsvファイルを全部読み込む関数
def read_raw_all_csv(target_path = '../input/'):
    for file in glob.glob(target_path+'*.csv'):
        raw_name = 'raw_' + file.replace(target_path,'').replace('.csv','')
        exec('{} = pd.read_csv(\'{}\')'.format(raw_name, file), globals())
        print('read {} as {}'.format(file, raw_name))

In [3]:
read_raw_all_csv()

read ../input/new_merchant_transactions.csv as raw_new_merchant_transactions
read ../input/test.csv as raw_test
read ../input/merchants.csv as raw_merchants
read ../input/train.csv as raw_train
read ../input/sample_submission.csv as raw_sample_submission


In [4]:
raw_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [5]:
raw_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [17]:
# 同じcard_idが複数名いるか確認
print('number of row in raw_train is {}. number of unique card_id is {}'.format(raw_train.shape[0], len(raw_train['card_id'].unique())))
print('number of row in raw_test is {}. number of unique card_id is {}'.format(raw_test.shape[0], len(raw_test['card_id'].unique())))

number of row in raw_train is 201917. number of unique card_id is 201917
number of row in raw_test is 123623. number of unique card_id is 123623


### 考察
 * trainとtestの比率は5:3くらい
 * train、testともにcard_idに重複はない

In [19]:
# trainとtestにcard_idの重複があるか確認


In [20]:
raw_new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29
